<a href="https://colab.research.google.com/github/meetmehedi/SaveFood/blob/main/Food_extra.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tensorflow tensorflow-datasets pillow

In [2]:
import os
import json
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers


In [3]:
IMG_SIZE = 224
BATCH_SIZE = 32
EPOCHS = 6   # Increase for better accuracy
os.makedirs("models", exist_ok=True)

In [4]:
def preprocess(example):
    image = tf.image.resize(example['image'], (IMG_SIZE, IMG_SIZE))
    image = tf.cast(image, tf.float32) / 255.0
    label = example['label']
    return image, label


In [5]:
def prepare_datasets():
    print("📥 Downloading Food-101 dataset...")
    ds_train, ds_info = tfds.load(
        'food101', split='train',
        with_info=True, shuffle_files=True,
        as_supervised=False
    )
    ds_test = tfds.load(
        'food101', split='validation',
        as_supervised=False
    )

    num_classes = ds_info.features['label'].num_classes
    print(f"✅ Found {num_classes} classes.")

    train = ds_train.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    train = train.shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    val = ds_test.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
    val = val.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

    return train, val, num_classes, ds_info


In [6]:
def build_model(num_classes):
    base = tf.keras.applications.MobileNetV2(
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
        include_top=False,
        weights='imagenet'
    )
    base.trainable = False  # Freeze base for faster training

    inputs = tf.keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = base(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = tf.keras.Model(inputs, outputs)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    return model

In [ ]:
train_ds, val_ds, n_classes, ds_info = prepare_datasets()
model = build_model(n_classes)
model.summary()

model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS)

# Save model
model_path = "models/food101_mobilenetv2.h5"
model.save(model_path)

# Save label names
label_names = ds_info.features['label'].names
with open("models/label_names.json", "w") as f:
    json.dump(label_names, f)

print(f"💾 Model saved to {model_path}")
print("💾 Labels saved to models/label_names.json")



📥 Downloading Food-101 dataset...


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

In [ ]:

from google.colab import files
files.download("models/food101_mobilenetv2.h5")
files.download("models/label_names.json")